In [238]:
import scipy
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [237]:
def convertDiffuseFile(fileInName):   
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['covis'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [239]:
def convertImagingFile(fileInName):
    fileContents = scipy.io.loadmat(fileInName)
    covisDict = {}
    covisDict['header'] = fileContents['__header__'].decode('utf-8')
    covisDict['version'] = fileContents['__version__']
    covisDict['globals'] = fileContents['__globals__']
    covis = fileContents['imaging'][0][0]
    for name in covis.dtype.names:
        covisDict[name] = covis[name]
    
    print(f'{fileInName} has been converted')
    return covisDict

In [240]:
def readCoords2D(fileInName):
    covisDict = convertDiffuseFile(fileInName)
    wList = [w for w in covisDict['grid'][0][0][0]['w']]
    xList = [x for x in covisDict['grid'][0][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0][0]['y']]
    vList = [v for v in covisDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'vList': vList, 'wList': wList}
    return coordsDict

In [241]:
def readCoords3D(fileInName, dataVal):
    covisDict = convertImagingFile(fileInName)
    xList = [x for x in covisDict['grid'][0][0]['x']]
    yList = [y for y in covisDict['grid'][0][0]['y']]
    zList = [z for z in covisDict['grid'][0][0]['z']]
    vList = [v for v in covisDict['grid'][0][0][f'{dataVal}']]
    wList = [w for w in covisDict['grid'][0][0]['w']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList, 'wList': wList}
    return coordsDict

In [254]:
def readCoordsBathymetry(bathyFileInName, diffuseFileInName):
    bathyDict = convertDiffuseFile(bathyFileInName)
    diffuseDict = convertDiffuseFile(diffuseFileInName)
    xList = [x for x in diffuseDict['grid'][0][0][0]['x']]
    yList = [y for y in diffuseDict['grid'][0][0][0]['y']]
    zList = [z for z in bathyDict['grid'][0][0]['v']]
    vList = [v for v in diffuseDict['grid'][0][0][0]['v']]
    coordsDict = {'xList': xList, 'yList': yList, 'zList': zList, 'vList': vList}
    return coordsDict

In [276]:
def createCoordsOfInterest(fileInName, opt1 = '2', opt2 = 'Ia', bathyFile = 0):
    if opt1 == '2':
        coordsDict = readCoords2D(fileInName)
        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

    elif opt1 == '3':
        coordsDict = readCoords3D(fileInName, opt2)
        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}

    elif opt1 == '4':
        coordsDict = readCoordsBathymetry(bathyFile, fileInName)
        coordsDict = {key: np.concatenate(coordsDict[key]).flatten() for key in coordsDict.keys()}
        for key in coordsDict.keys():
            coordsDict[key] = coordsDict[key][:len(coordsDict['zList'])]
    
    mask = coordsDict['vList'] != 0
    coordsDict = {key: coordsDict[key][mask] for key in coordsDict.keys()}
    return coordsDict

In [296]:
def plotDiffuse2DV1(fileInName):
    coordsOfInterest = createCoordsOfInterest(fileInName)
    plt.figure(figsize=(8, 6))
    flatV = np.concatenate([arr.flatten() for arr in coordsOfInterest['vList']])
    max = np.amax(flatV)
    min = np.amin(flatV) + 10e-10
    norm = mcolors.LogNorm(vmin = min, vmax = max)
    scatter = plt.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], c = coordsOfInterest['vList'], cmap = 'viridis', norm = norm)

    cbar = plt.colorbar(scatter)
    cbar.set_label('Data Values')

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(fileInName.split('\\')[-1])
    plt.show()


In [298]:
def plotDiffuse2DV2(fileInName):
    coordsOfInterest = createCoordsOfInterest(fileInName)
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], c=coordsOfInterest['vList'], cmap = 'viridis', vmin=0, vmax=0.00005)

    cbar = plt.colorbar(scatter)
    cbar.set_label('Data Values')

    plt.xlabel('East of COVIS (m)')
    plt.ylabel('North of COVIS (m)')
    plt.title(fileInName.split('\\')[-1])
    plt.show()

In [ ]:
plotDiffuse2DV1(r'matFiles\COVIS-20230701T003002-diffuse1.mat')
plotDiffuse2DV2(r'matFiles\COVIS-20230701T003002-diffuse1.mat')

In [302]:
def plotFullImaging3DV1(fileInName, opt2):
    coordsOfInterest = createCoordsOfInterest(fileInName, '3', opt2)
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))

    scatter = ax.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], coordsOfInterest['zList'], c = coordsOfInterest['vList'], cmap = 'viridis')
    
    cbar = plt.colorbar(scatter)
    cbar.set_label(f'{opt2} Values')
    cbar_ticks = np.arange(min, max + 1)
    cbar.set_ticks(cbar_ticks)

    ax.set_xlabel('East of COVIS (m)')
    ax.set_ylabel('North of COVIS (m)')
    ax.set_zlabel('Z Axis')
    ax.set_title(f'3D Scatter Plot (Logarithmic scale, {opt2})')

    plt.show()

In [ ]:
plotFullImaging3DV1(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia')
plotFullImaging3DV1(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Id')
plotFullImaging3DV1(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia_filt')
plotFullImaging3DV1(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Id_filt')
plotFullImaging3DV1(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Kp')

In [332]:
def plotFullImaging3DV2(fileInName, opt2):
    coordsOfInterest = createCoordsOfInterest(fileInName, '3', opt2)

    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))

    fig = go.Figure(data=go.Isosurface(
        x = coordsOfInterest['xList'],
        y = coordsOfInterest['yList'],
        z = coordsOfInterest['zList'],
        value = coordsOfInterest['vList'],
        colorscale  = 'Viridis',
        isomin = min,
        isomax = max,
        surface_count=100,
        caps=dict(x_show=False, y_show=False, z_show=False)
    ))

    fig.update_layout(
        title=f'3D Isosurface Plot (Logarithmic scale, {opt2})',
        scene=dict(
            xaxis_title='East of COVIS (m)',
            yaxis_title='North of COVIS (m)',
            zaxis_title='Z Axis',
        ),
        coloraxis_colorbar=dict(
            title=f'{opt2} Values',
            tickvals=np.linspace(min, max, num=5),
            ticktext=[f'{val:.2e}' for val in np.linspace(min, max, num=5)]
        )
    )

    fig.show()


In [ ]:
plotFullImaging3DV2(r'matFiles\COVIS-20230101T000002-imaging1.mat', 'Ia')

In [304]:
def plotDiffuseBathymetry(fileInName):
    coordsOfInterest = createCoordsOfInterest(fileInName, '4', bathyFile = r'matFiles\covis_bathy_2019b.mat')
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    coordsOfInterest['zList'] = np.nan_to_num(np.array(coordsOfInterest['zList']))
    for key in coordsOfInterest.keys():
        coordsOfInterest[key] = coordsOfInterest[key][:len(coordsOfInterest['zList'])]

    coordsOfInterest['vList'] = np.log10(coordsOfInterest['vList'])
    max = np.floor(np.amax(coordsOfInterest['vList']))
    min = np.ceil(np.amin(coordsOfInterest['vList']))

    scatter = ax.scatter(coordsOfInterest['xList'], coordsOfInterest['yList'], coordsOfInterest['zList'], c = coordsOfInterest['vList'], cmap = 'viridis')

    cbar = plt.colorbar(scatter)
    cbar.set_label('Values')
    cbar_ticks = np.arange(min, max + 1)
    cbar.set_ticks(cbar_ticks)

    ax.set_xlabel('East of COVIS (m)')
    ax.set_ylabel('North of COVIS (m)')
    ax.set_zlabel('Z Axis')
    ax.set_title(f'Diffuse Draped over Bathymetry (Logarithmic scale)')

    plt.show()

In [ ]:
plotDiffuseBathymetry(r'matFiles\COVIS-20230701T003002-diffuse1.mat')

In [324]:
def createHistogram(fileInName, opt1, bins = 20):
    coordsOfInterest = createCoordsOfInterest(fileInName, opt1)
    plt.figure(figsize=(10, 6))
    plt.hist(coordsOfInterest['vList'], bins=bins, edgecolor='black', alpha=0.7)
    plt.yscale('log')

    
    plt.title('Distribution of values')
    plt.xlabel('Values')
    plt.ylabel('Counts')
    
    plt.show()

In [ ]:
createHistogram(r'matFiles\COVIS-20230701T003002-diffuse1.mat', opt1 = '2', bins = 50)